In [1]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict

from random import *

In [2]:
vertex_set = set();
vertex_dict = {}
vertex_followlist ={}
sink_dict = {};
sink_followedlist = defaultdict(list)

with open("train.txt") as trainfile:
    for i, line in enumerate(trainfile):
        line_list = [int(k) for k in line[:-1].split("\t")];
        vertex_dict[line_list[0]]=len(line_list)-1
        vertex_set.add(line_list[0]);
        vertex_followlist[line_list[0]]= line_list[1:]
        for s in line_list[1:]:
            sink_followedlist[s].append(line_list[0])
            if s in sink_dict:
                sink_dict[s] += 1;
            else:
                sink_dict[s] = 1;
        if i % 1000 == 0:
            print(i);

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [3]:
print(sink_dict[4867135]);
print(len(vertex_set));

1
20000


In [4]:
new_sink_dict = {};
threshold = 10;
for k in sink_dict:
    if sink_dict[k] >= threshold:
        new_sink_dict[k] = sink_dict[k];
        
new_sink_set = set(new_sink_dict);
print(len(new_sink_set))

394342


In [5]:
test_vertex_and_sink_set = set();

with open("test-public.txt") as testfile:
    for i, line in enumerate(testfile):
        if i == 0:
            continue;
        line_list = [int(k) for k in line[:-1].split("\t")];
        for s in line_list:
            test_vertex_and_sink_set.add(s);
print(len(test_vertex_and_sink_set));

total_set = test_vertex_and_sink_set.union(new_sink_set).union(vertex_set);
print(len(total_set));

5947
401218


In [6]:
total_dict = {};
total_list = []
for i, p in enumerate(total_set):
    total_dict[p] = i
    total_list.append(p)

In [25]:
max_neighbors = 1000

import numpy as np
total_array = np.array(total_list);

pairs = [];
true_pairs = [];
"""
"""

with open("train.txt") as trainfile:
    for i, line in enumerate(trainfile):
        line_list = [int(k) for k in line[:-1].split("\t")];
        v = line_list[0];
        ranking = [-sink_dict[k] for k in line_list[1:]];
        sorting = np.argsort(ranking);
        filtered_linelist = np.array(line_list[1:])[sorting];
        for s in filtered_linelist[1:max_neighbors]:
            if s in total_set:
                pairs.append([total_dict[v], total_dict[s]]);
        if i % 1000 == 0:
            print(i);

test_pairs = [];
with open("test-public.txt") as testfile:
    for i, line in enumerate(testfile):
        if i == 0:
            continue;
        line_list = [int(k) for k in line[:-1].split("\t")];
        test_pairs.append([line_list[0], total_dict[line_list[1]], total_dict[line_list[2]]]);
        
correspondence = total_array

a=0
for i in pairs:
    true_pairs.append([correspondence[i[0]],correspondence[i[1]]])



0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


NameError: name 'true_testpairs' is not defined

In [28]:
true_testpairs = []
for i in test_pairs:
    true_testpairs.append([correspondence[i[1]],correspondence[i[2]]])
print(true_testpairs[0])

[2184483, 1300190]


In [8]:
print (len(true_pairs))

print (true_pairs[500000])
print (true_pairs[5000000])
print (len(test_pairs))

5168055
[1006458, 599443]
[1938554, 3364808]
2000


In [9]:
np.savez_compressed("filtered_data", correspondence = total_array, pairs = pairs, test_pairs = test_pairs);

In [10]:
new_vertex_followlist = defaultdict(list)
new_sink_followlist = defaultdict(list)
new_vertex_dict = {}
new_sink_dict = {}
for i in true_pairs:
    new_vertex_followlist[i[0]].append(i[1])
    new_sink_followlist[i[1]].append(i[0])

for i in new_vertex_followlist.keys():
    new_vertex_dict[i]=len(new_vertex_followlist[i])
    for j in new_vertex_followlist[i]:
        if j in new_sink_dict:
            new_sink_dict[j]+=1
        else:   
            new_sink_dict[j]=1
print (len(new_vertex_followlist))
print (len(new_sink_followlist))
print (len(new_vertex_dict))
print (len(new_sink_dict))

19138
319312
19138
319312


In [11]:
# test
def findneighbor(a,b):
    import math

#     c is common friend
#     1.a→c→b
#     2.a→c←b
#     3.a←c→b
#     4.a←c←b
#     5. Jaccard's coeffcient common friends/total friends
#     6. Adamic/Adar 

    tempA,tempB = [],[]
    count1,count2,count3,count4,count5,count6,count7 = 0,0,0,0,0,0,0
#     a 一定在 vertex_set
    if a in new_vertex_followlist:
        tempA = new_vertex_followlist[a]
    if b in new_vertex_followlist:
        tempB = new_vertex_followlist[b]
    
    if tempA !=None and b in new_sink_followlist:
            count1=len(set(tempA).intersection(new_sink_followlist[b]))
  
    count2 = len(set(tempA).intersection(tempB))
    if a in new_sink_followlist and b in new_sink_followlist:
        count3 = len(set(new_sink_followlist[a]).intersection(new_sink_followlist[b]))
        
    if a in new_sink_followlist and tempB!=None:
        count4 = len(set(new_sink_followlist[a]).intersection(tempB))
    
#     if tempA !=None:
#         for i in tempA:
#             if i in new_vertex_followlist:
#                 if b in new_vertex_followlist[i]:
#                     count1+=1
    
#     count2 = len(set(tempA).intersection(tempB))
#     count3 = len(set(new_sink_followlist[a]).intersection(new_sink_followlist[b]))
    
#     if tempB != None:
#         for i in tempB:
#             if i in new_vertex_followlist:
#                 if a in new_vertex_followlist[i]:
#                     count4+=1
                    
    totalA = set().union(new_sink_followlist[a],tempA)
    totalB = set().union(new_sink_followlist[b],tempB)
    commonFriends = list(set(totalA).intersection(totalB))
    total_AB = len(totalA.union(totalB))
    if total_AB!=0:
        count5 = len(commonFriends)/total_AB
    
#     for commonFriend in commonFriends:
#         if commonFriend in new_vertex_followlist:
#             freq_comfri=new_vertex_dict[commonFriend]+new_sink_dict[commonFriend]
#         else: 
#             freq_comfri=new_sink_dict[commonFriend]
#         count6+=1/math.log(freq_comfri)

    
    return (count1,count2,count3,count4,count5)
findneighbor(1810899,2882556)

(3, 7, 5, 5, 0.004608294930875576)

In [12]:
c =0
show=0
for i in new_vertex_dict.keys():
    show+=1
    if i in new_sink_dict:
        c+=1
print (c)

# 19136/19138

19136


In [14]:
# # original one
# def countFollowingFollower(a,b):
#     """return 
#     1. a following count 
#     2. a followed count
#     3. b following count
#     4. b followed count
#     5. a following/a followed
#     6. b following/b followed
#     7. if b following a
#     8. if a following b (y=label)

#     """
#     count1,count2,count3,count4,count5,count6,count7,count8=0,0,0,0,0,0,0,0
#     if a in vertex_set:
#         count1 = vertex_dict[a]
#     if b in vertex_set:
#         count3 = vertex_dict[b]

#     count2 = sink_dict[a]
#     count4 = sink_dict[b]
    
#     if count2!=0:
#         count5 = count1/count2
#     else:
#         count5 = count1
#     if count4!=0:
#         count6 = count3/count4
#     else:
#         count6 = count2
#     if b in vertex_set:    
#         if a in new_vertex_followlist[b]:
#             count7=1
# #     if a in vertex_set:    
# #         if b in new_vertex_followlist[a]:
# #             count8=1
#     return (count1,count2,count3,count4,count5,count6,count7,count8)


# """
# count8  if a follow b


# countFollowingFollower(1406432,2481036)

In [18]:
def countFollowingFollower(a,b):
    """return 
    1. a following count 
    2. a followed count
    3. b following count
    4. b followed count
    5. a following/a followed
    6. b following/b followed
    7. if b following a
    8. if a following b (y=label)

    """
    count1,count2,count3,count4,count5,count6,count7,count8=0,0,0,0,0,0,0,0
    
    
    if a in new_vertex_dict:
        count1 = new_vertex_dict[a]
    if b in new_vertex_dict:
        count3 = new_vertex_dict[b]
    if a in new_sink_dict:
        count2 = new_sink_dict[a]
    if b in new_sink_dict:
        count4 = new_sink_dict[b]
    
    if count2!=0:
        count5 = count1/count2
    else:
        count5 = count1
    if count4!=0:
        count6 = count3/count4
    else:
        count6 = count2
    if b in new_vertex_followlist:    
        if a in new_vertex_followlist[b]:
            count7=1
#     if a in vertex_set:    
#         if b in new_vertex_followlist[a]:
#             count8=1
    return (count1,count2,count3,count4,count5,count6,count7)

"""
count8  if a follow b

"""
countFollowingFollower(1406432,2481036)

(63, 6, 13, 23, 10.5, 0.5652173913043478, 0)

In [50]:
input_train = []
count=0
for i in true_pairs:
    a=countFollowingFollower(i[0],i[1])
#     b=findneighbor(i[0],i[1])
#     print (b)
    
    count+=1
    input_train.append(list(a))
    if count%500000==0:
        print (count)


print (len(input_train))

500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5168055


In [51]:
input_train2 = []
count=0
for i in true_pairs:
#     a=countFollowingFollower(i[0],i[1])
    b=findneighbor(i[0],i[1])

    count+=1
    input_train2.append(list(b))
    if count%5000==0:
        print (count)

print (len(input_train2))

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000
470000
475000
480000
485000
490000
495000
500000
505000
510000
515000
520000
525000
530000
535000
540000
545000
550000
555000
560000
565000
570000
575000
580000
585000
590000
595000
600000
605000
610000
615000
620000
625000
630000
635000
640000
645000
650000
655000
660000
665000
670000
675000
680000
685000
690000
695000
700000
705000
710000
715000
720000
725000
73000

In [ ]:
# sink_sum = set(sink_dict)
# all_set = sink_sum.union(vertex_set)
# sum_ = len(all_set)-1
# all_set_list = list(all_set)
# false_pairs = []
# for i in range (5173000):
#     random1 = randint(0, sum)
#     random2 = randint(0,sum)
#     false_pairs.append([all_set_list[random1],all_set_list[random2]])


In [ ]:
# count=0    
# for i in false_pairs:
#     a=countFollowingFollower(i[0],i[1])
#     b=findneighbor(i[0],i[1])
#     count+=1
#     if count % 100000 == 0:
#         print(count);
    
#     input_train.append([b[0],b[1],b[2],b[3],b[4],b[5],a[0],a[1],a[2],a[3],a[4],a[5],a[6],a[7]])
    
# print (len(input_train))
    
# niubi = np.array(input_train)

In [52]:
temptest = np.hstack((input_train,input_train2))

In [53]:
print (len(temptest[0]))

12


In [69]:
true_label = np.ones(len(temptest))
true_featurewithlabel = np.insert(temptest, 12, values=true_label, axis=1)

In [70]:
print (len(true_featurewithlabel))

5168055


In [63]:
total_list = list(total_set)
sum = len(total_list)-1
false_pairs = []
for i in range(5168055):
    random1 = randint(0,sum)
    random2 = randint(0,sum)
    if random1 in new_vertex_followlist and random2 in new_vertex_followlist[random1]:
        continue
    false_pairs.append([total_list[random1],total_list[random2]])
print (len(false_pairs))

5168055


In [67]:
input_train_1 = []
count=0
for i in false_pairs:
    a=countFollowingFollower(i[0],i[1])
#     b=findneighbor(i[0],i[1])
#     print (b)
    
    count+=1
    input_train_1.append(list(a))
    if count%500000==0:
        print (count)


print (len(input_train_1))

500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5168055


In [71]:
input_train_2 = []
count=0
for i in false_pairs:
#     a=countFollowingFollower(i[0],i[1])
    b=findneighbor(i[0],i[1])

    count+=1
    input_train_2.append(list(b))
    if count%5000==0:
        print (count)

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000
370000
375000
380000
385000
390000
395000
400000
405000
410000
415000
420000
425000
430000
435000
440000
445000
450000
455000
460000
465000
470000
475000
480000
485000
490000
495000
500000
505000
510000
515000
520000
525000
530000
535000
540000
545000
550000
555000
560000
565000
570000
575000
580000
585000
590000
595000
600000
605000
610000
615000
620000
625000
630000
635000
640000
645000
650000
655000
660000
665000
670000
675000
680000
685000
690000
695000
700000
705000
710000
715000
720000
725000
73000

In [72]:
false_temptest = np.hstack((input_train_1,input_train_2))
print(len(false_temptest[0]))

12


In [73]:
false_label = np.zeros(len(false_temptest))
false_featurewithlabel = np.insert(false_temptest, 12, values=false_label, axis=1)
print (len(false_featurewithlabel))

5168055


In [74]:
together = np.vstack((true_featurewithlabel,false_featurewithlabel))
print(np.shape(together))

(10336110, 13)


In [75]:
np.savez_compressed("result_1000", result = together);

In [30]:
test_output = []
for i in true_testpairs:
    temp =[i[0],i[1]]
    test_output.append(temp)
print(np.shape(test_output))

(2000, 2)


In [35]:
test_features = []
for i in test_output:
    a=countFollowingFollower(i[0],i[1])
    b=findneighbor(i[0],i[1])
    test_features.append(list(a)+list(b))
print (np.shape(test_features))

(2000, 12)


In [34]:
np.savez_compressed("test_features", testresult = test_features);

In [36]:
print (test_features)

[[70, 84, 0, 0, 0.8333333333333334, 84, 0, 0, 0, 0, 0, 0.0], [289, 26, 0, 142, 11.115384615384615, 0.0, 0, 0, 0, 0, 0, 0.0], [155, 3, 0, 0, 51.666666666666664, 3, 0, 0, 0, 0, 0, 0.0], [63, 6, 13, 23, 10.5, 0.5652173913043478, 0, 4, 4, 1, 0, 0.06818181818181818], [161, 12, 49, 146, 13.416666666666666, 0.3356164383561644, 0, 0, 5, 0, 0, 0.014124293785310734], [7, 12, 296, 891, 0.5833333333333334, 0.33221099887766553, 1, 7, 7, 11, 10, 0.01253616200578592], [12, 2, 0, 482, 6.0, 0.0, 0, 1, 0, 0, 0, 0.0020242914979757085], [56, 20, 0, 15, 2.8, 0.0, 0, 1, 0, 1, 0, 0.012987012987012988], [315, 11, 0, 8, 28.636363636363637, 0.0, 0, 0, 0, 0, 0, 0.0], [22, 1, 0, 1, 22.0, 0.0, 0, 0, 0, 0, 0, 0.0], [45, 5, 0, 25, 9.0, 0.0, 0, 0, 0, 0, 0, 0.0], [55, 8, 0, 0, 6.875, 8, 0, 0, 0, 0, 0, 0.0], [235, 44, 0, 2, 5.340909090909091, 0.0, 0, 0, 0, 0, 0, 0.0], [43, 7, 0, 1, 6.142857142857143, 0.0, 0, 0, 0, 0, 0, 0.0], [142, 2, 579, 21, 71.0, 27.571428571428573, 0, 2, 3, 0, 0, 0.006830601092896175], [71, 4, 0, 1